# Neural Machine Translation (NMT)

This notebook is adapted from https://pytorch.org/hub/pytorch_fairseq_translation/.

### Requirements

In [1]:
!apt-get update && apt-get -y install gcc g++

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease           
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Hit:5 https://deb.nodesource.com/node_14.x focal InRelease                     
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Fetched 271 kB in 33s (8,193 B/s)  
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
gcc is already the newest version (4:9.3.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 144 not upgraded.


In [2]:
pip install bitarray fastBPE hydra-core omegaconf regex requests sacremoses subword_nmt scikit-learn sacrebleu Cython

Note: you may need to restart the kernel to use updated packages.


### English-to-French Translation

In [3]:
import torch

# Load an En-Fr Transformer model trained on WMT'14 data :
en2fr = torch.hub.load('pytorch/fairseq', 'transformer.wmt14.en-fr', tokenizer='moses', bpe='subword_nmt')

# Use the GPU (optional):
en2fr.cuda()

# Translate with beam search:
fr = en2fr.translate('Hello world!', beam=5)
assert fr == 'Bonjour à tous !'

# Manually tokenize:
en_toks = en2fr.tokenize('Hello world!')
assert en_toks == 'Hello world !'

# Manually apply BPE:
en_bpe = en2fr.apply_bpe(en_toks)
assert en_bpe == 'H@@ ello world !'

# Manually binarize:
en_bin = en2fr.binarize(en_bpe)
assert en_bin.tolist() == [329, 14044, 682, 812, 2]

# Generate five translations with top-k sampling:
fr_bin = en2fr.generate(en_bin, beam=5, sampling=True, sampling_topk=20)
assert len(fr_bin) == 5

# Convert one of the samples to a string and detokenize
fr_sample = fr_bin[0]['tokens']
fr_bpe = en2fr.string(fr_sample)
fr_toks = en2fr.remove_bpe(fr_bpe)
fr = en2fr.detokenize(fr_toks)
assert fr == en2fr.decode(fr_sample)

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_fairseq_master


[2023-08-22 11:09:26,451] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt14.en-fr.joined-dict.transformer.tar.bz2 HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt14.en-fr.joined-dict.transformer.tar.bz2 from cache at /home/jovyan/.cache/torch/pytorch_fairseq/53f403ba27ab138b06c1a8d78f5bb4f1722567ac3d3b3e41f821ec2cae2974da.7ef8ab763efda16d3c82dd8b5a574bdfe524e078bac7b444ea1a9c5d355b55ae
/opt/conda/lib/python3.8/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/opt/conda/lib/python3.8/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please sp

In [4]:
print(fr)

Bonjour à tous !


### English-to-German Translation

In [5]:
import torch

# Load an En-De Transformer model trained on WMT'19 data:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')

# Access the underlying TransformerModel
assert isinstance(en2de.models[0], torch.nn.Module)

# Translate from En-De
de = en2de.translate('PyTorch Hub is a pre-trained model repository designed to facilitate research reproducibility.')
assert de == 'PyTorch Hub ist ein vorgefertigtes Modell-Repository, das die Reproduzierbarkeit der Forschung erleichtern soll.'

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_fairseq_master
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /home/jovyan/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq.tasks.translation:[en] dictionary: 42024 types
INFO:fairseq.tasks.translation:[de] dictionary: 42024 types
INFO:fairseq.models.fairseq_model:{'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': 'simpl

In [6]:
print(de)

PyTorch Hub ist ein vorgefertigtes Modell-Repository, das die Reproduzierbarkeit der Forschung erleichtern soll.


### A round-trip translation to create a paraphrase

In [7]:
# Round-trip translations between English and German:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = de2en.translate(en2de.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch Hub is a fantastic interface!'

# Compare the results with English-Russian round-trip translation:
en2ru = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-ru.single_model', tokenizer='moses', bpe='fastbpe')
ru2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.ru-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = ru2en.translate(en2ru.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch is a great interface!'

Using cache found in /home/jovyan/.cache/torch/hub/pytorch_fairseq_master
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dl.fbaipublicfiles.com:443
DEBUG:urllib3.connectionpool:https://dl.fbaipublicfiles.com:443 "HEAD /fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz HTTP/1.1" 200 0
INFO:fairseq.file_utils:loading archive file https://dl.fbaipublicfiles.com/fairseq/models/wmt19.en-de.joined-dict.single_model.tar.gz from cache at /home/jovyan/.cache/torch/pytorch_fairseq/81a0be5cbbf1c106320ef94681844d4594031c94c16b0475be11faa5a5120c48.63b093d59e7e0814ff799bb965ed4cbde30200b8c93a44bf8c1e5e98f5c54db3
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq.tasks.translation:[en] dictionary: 42024 types
INFO:fairseq.tasks.translation:[de] dictionary: 42024 types
INFO:fairseq.models.fairseq_model:{'_name': None, 'common': {'_name': None, 'no_progress_bar': True, 'log_interval': 100, 'log_format': 'simpl

In [8]:
print(paraphrase)

PyTorch is a great interface!
